In [10]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DBCONN")

In [11]:
import psycopg

load_dotenv()
dbconn = os.getenv("DBCONN")

# establish database connection using connection string from .env
conn = psycopg.connect(dbconn)

# create a cursor
cur = conn.cursor()



In [12]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS btc_data_news(
        id INTEGER PRIMARY KEY,
        title TEXT,
        author TEXT,
        link TEXT,
        published_date DATE
    );
''')

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.rmhwmflqmrpurnexvvgh.supabase.co database=postgres) at 0x119a6b650>

In [13]:
cur.execute("""
        create table if not exists alpha_vantage_daily(
          trade_date date primary key,
          open  numeric(12,6),
          high  numeric(12,6),
          low   numeric(12,6),
          close numeric(12,6),
          volume bigint
        );
        """)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=db.rmhwmflqmrpurnexvvgh.supabase.co database=postgres) at 0x119a6b650>

In [14]:
conn.commit()

In [ ]:
import pandas as pd
import datetime as dt

df = pd.read_csv("btc_data_news.csv", parse_dates=["Dates.N"])

conn = psycopg.connect(dbconn)
cur = conn.cursor()

for _, row in df.iterrows():
    # Skip rows with missing data
    if pd.isna(row["Dates.N"]):
        continue

    # Convert to Python datetime (if not already)
    published_date = pd.to_datetime(row["Dates.N"]).to_pydatetime()
    title = row["Title"]
    author = row["Author"]
    id_ = int(row["ID"])

    cur.execute(
        '''
        INSERT INTO btc_data_news (id, title, author, published_date)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (id) DO UPDATE SET
            title = EXCLUDED.title,
            author = EXCLUDED.author,
            published_date = EXCLUDED.published_date;
        ''',
        [id_, title, author, published_date]
    )

conn.commit()
cur.close()
conn.close()

In [18]:
df = pd.read_csv("alpha_vantage.csv", parse_dates=["date"])

conn = psycopg.connect(dbconn)
cur = conn.cursor()

for _, row in df.iterrows():
    if pd.isna(row["date"]):
        continue

    cur.execute(
        '''
        INSERT INTO alpha_vantage_daily (trade_date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (trade_date) DO UPDATE SET
            open = EXCLUDED.open,
            high = EXCLUDED.high,
            low = EXCLUDED.low,
            close = EXCLUDED.close,
            volume = EXCLUDED.volume;
        ''',
        [
            row["date"].date(),
            row["open"],
            row["high"],
            row["low"],
            row["close"],
            int(row["volume"])
        ]
    )

conn.commit()
cur.close()
conn.close()